In [52]:
import pandas, numpy, indo_processor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [2]:
data = pandas.read_csv(r"dataset\20191002-reviews.csv")

data.head()

,itemId,category,name,rating,originalRating,reviewTitle,reviewContent,likeCount,upVotes,downVotes,helpful,relevanceScore,boughtDate,clientType,retrievedDate
0,100002528,beli-harddisk-eksternal,Kamal U.,5,NaN,NaN,bagus mantap dah sesui pesanan,0,0,0,True,26.51,09 Apr 2019,androidApp,2019-10-02
1,100002528,beli-harddisk-eksternal,yofanca m.,4,NaN,NaN,"Bagus, sesuai foto",0,0,0,True,22.49,24 Sep 2017,androidApp,2019-10-02
2,100002528,beli-harddisk-eksternal,Lazada Customer,5,NaN,ok mantaaapppp barang sesuai pesanan.. good,okkkkk mantaaaaaaapppp ... goood,0,0,0,True,21.50,04 Apr 2018,androidApp,2019-10-02
3,100002528,beli-harddisk-eksternal,Lazada Customer,4,NaN,NaN,bagus sesuai,0,0,0,True,20.51,22 Sep 2017,androidApp,2019-10-02
4,100002528,beli-harddisk-eksternal,Yosep M.,5,NaN,NaN,NaN,0,0,0,True,16.01,17 Agu 2018,androidApp,2019-10-02


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203787 entries, 0 to 203786
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   itemId          203787 non-null  int64  
 1   category        203787 non-null  object 
 2   name            203787 non-null  object 
 3   rating          203787 non-null  int64  
 4   originalRating  8 non-null       float64
 5   reviewTitle     23399 non-null   object 
 6   reviewContent   107029 non-null  object 
 7   likeCount       203787 non-null  int64  
 8   upVotes         203787 non-null  int64  
 9   downVotes       203787 non-null  int64  
 10  helpful         203787 non-null  bool   
 11  relevanceScore  203787 non-null  float64
 12  boughtDate      196680 non-null  object 
 13  clientType      203787 non-null  object 
 14  retrievedDate   203787 non-null  object 
dtypes: bool(1), float64(2), int64(5), object(7)
memory usage: 22.0+ MB


In [4]:
data.describe()

,itemId,rating,originalRating,likeCount,upVotes,downVotes,relevanceScore
count,2.037870e+05,203787.000000,8.0,203787.000000,203787.000000,203787.000000,203787.000000
mean,2.836479e+08,4.603238,1.0,0.668634,0.668634,0.164638,25.097394
std,1.726207e+08,0.991164,0.0,12.192433,12.192433,2.013273,9.602434
min,6.068000e+03,1.000000,1.0,-1.000000,-1.000000,0.000000,0.960000
25%,1.600086e+08,5.000000,1.0,0.000000,0.000000,0.000000,18.010000
50%,3.541160e+08,5.000000,1.0,0.000000,0.000000,0.000000,23.510000
75%,4.147421e+08,5.000000,1.0,0.000000,0.000000,0.000000,29.670000
max,7.242170e+08,5.000000,1.0,1776.000000,1776.000000,111.000000,76.500000


In [5]:
used_data = data[["itemId", "rating", "reviewContent"]]

used_data.describe()

,itemId,rating
count,2.037870e+05,203787.000000
mean,2.836479e+08,4.603238
std,1.726207e+08,0.991164
min,6.068000e+03,1.000000
25%,1.600086e+08,5.000000
50%,3.541160e+08,5.000000
75%,4.147421e+08,5.000000
max,7.242170e+08,5.000000


In [6]:
used_data

,itemId,rating,reviewContent
0,100002528,5,bagus mantap dah sesui pesanan
1,100002528,4,"Bagus, sesuai foto"
2,100002528,5,okkkkk mantaaaaaaapppp ... goood
3,100002528,4,bagus sesuai
4,100002528,5,NaN
...,...,...,...
203782,9467899,5,barang bagus dengan harga kompetitif
203783,9467899,3,NaN
203784,9548087,5,NaN
203785,9548767,3,Ok.thank lazada Pagi order sore barang nyampe ...


In [7]:
used_data = used_data.drop_duplicates()
used_data = used_data.dropna()

In [8]:
used_data

,itemId,rating,reviewContent
0,100002528,5,bagus mantap dah sesui pesanan
1,100002528,4,"Bagus, sesuai foto"
2,100002528,5,okkkkk mantaaaaaaapppp ... goood
3,100002528,4,bagus sesuai
7,100002528,1,bima
...,...,...,...
82221,9973500,3,"Pengiriman lebih cepat , packing rapih Dan ba..."
87955,687840070,1,I can't login to activated this netbook becaus...
103881,377474276,4,"barang ok, pengiriman super lambat dn trpaksa ..."
195817,377474276,5,"mantapp,, terima kasih lazadaaa,, tv coocaa 40..."


In [9]:
used_data_coded = used_data

used_data.loc[used_data["rating"] >= 4, "rating"] = "good"
used_data.loc[used_data["rating"] != 'good', "rating"] = "bad"

used_data_coded

C:\Users\Atasha\AppData\Local\Temp\ipykernel_9724\2692551241.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'good' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  used_data.loc[used_data["rating"] >= 4, "rating"] = "good"


,itemId,rating,reviewContent
0,100002528,good,bagus mantap dah sesui pesanan
1,100002528,good,"Bagus, sesuai foto"
2,100002528,good,okkkkk mantaaaaaaapppp ... goood
3,100002528,good,bagus sesuai
7,100002528,bad,bima
...,...,...,...
82221,9973500,bad,"Pengiriman lebih cepat , packing rapih Dan ba..."
87955,687840070,bad,I can't login to activated this netbook becaus...
103881,377474276,good,"barang ok, pengiriman super lambat dn trpaksa ..."
195817,377474276,good,"mantapp,, terima kasih lazadaaa,, tv coocaa 40..."


In [10]:
used_data_coded = used_data_coded.sample(frac=1, random_state=69420)
used_data_coded = used_data_coded.reset_index()

used_data_coded

,index,itemId,rating,reviewContent
0,15051,160020846,good,barang sudah sampai paking y rapi mksih lazada
1,31469,278127,good,barang sesuai pesanan
2,11005,140953986,good,Barang sudah diterima tepat waktu. Kualitas ba...
3,48373,389216525,good,Pembelian kedua...tidak secepat pembelian pert...
4,68812,498408088,good,bagus bagat barang ya gan 😁 😀
...,...,...,...,...
41544,38430,354115963,good,sesuai pesanan
41545,39962,357431547,good,bagus.
41546,20273,160032830,good,barang bagus hanya saja untuk microsft office ...
41547,74785,5975849,good,mantapz


In [11]:
good_UDC = used_data_coded[used_data_coded["rating"] == "good"]
bad_UCD  = used_data_coded[used_data_coded["rating"] == "bad"]

good_UDC_under = good_UDC[:bad_UCD.shape[0]]

new_data = pandas.concat([good_UDC_under, bad_UCD]).sample(frac=1, random_state=69420)
new_data = new_data.reset_index().drop(["level_0", "index"], axis=1)

print(f"""
{good_UDC_under.shape}
{bad_UCD       .shape}
""")

new_data


(5172, 4)
(5172, 4)



,itemId,rating,reviewContent
0,108702758,good,pesanan sudah saya terima dengan baik...dan sa...
1,368162161,bad,ngk tw kenapa mngkn krna wkt pas pengirimannya...
2,121244218,good,Good trusted seller
3,160030643,bad,"lhaaa,,, saya berulang kali mesen dengan alama..."
4,160022809,bad,Pesanan informasi dari lazada barang telah dik...
...,...,...,...
10339,390739847,good,Barang bagus
10340,6137822,bad,Tolong dikirim kembali kok tiba tiba ada pesan...
10341,533174374,bad,Barang sangat jelek
10342,618142664,bad,data langsung error....harga yg berbicara


In [27]:
review = new_data["reviewContent"].apply(lambda x: indo_processor.process(x))

In [48]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(review)
Y = new_data["rating"]

X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 105026 stored elements and shape (10344, 5000)>

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=69420)

In [50]:

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [51]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8376027066215563

Classification Report:
               precision    recall  f1-score   support

         bad       0.84      0.84      0.84      1051
        good       0.84      0.83      0.83      1018

    accuracy                           0.84      2069
   macro avg       0.84      0.84      0.84      2069
weighted avg       0.84      0.84      0.84      2069



In [53]:
joblib.dump(model, "classifier.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']